# Developing custom metrics to use in NN

## Importing libraries

In [1]:
import pandas as pd
import numpy as np
from ipynb.fs.full.Useful_funcs import data_pipeline, compile_train, pre_model, create_huber # Custom funcs for data processing, modelling, compiling and training
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import selu
from tensorflow.keras.initializers import lecun_normal
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.losses import mse

## Loading dataset

In [2]:
housing = fetch_california_housing()

In [3]:
x_train, x_train_scaled, x_valid, x_valid_scaled, x_test, x_test_scaled, y_train, y_valid, y_test = data_pipeline(housing)

## Model

- The huber loss func defined earlier can be used as a custom metric.

In [5]:
input_shape = x_train.shape[1:]

In [6]:
pre_model()
model = Sequential()
model.add(Dense(30, activation = selu, kernel_initializer = lecun_normal(), input_shape = input_shape))
model.add(Dense(1))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                270       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [12]:
history = compile_train(model, (x_train_scaled, y_train), (x_valid_scaled, y_valid), mse, Nadam(), create_huber(2.), 5)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected EOF while parsing (<unknown>, line 1)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected EOF while parsing (<unknown>, line 1)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
363/363 [==============================] - 0s 1ms/step - loss: 2.0982 - huber_loss: 0.9192 - val_loss: 9.0831 - val_huber_loss: 0.4418
Epoch 2/5
363/363 [==============================] - 0s 940us/step - loss: 0.6052 - huber_loss: 0.2733 - val_loss: 5.8943 - val_huber_loss: 0.3463
Epoch 3/5
363/363 [==============================] - 0s 1ms/step - loss: 0.5113 - huber_

- If we use the same func as loss and metric, we can find that both gives different results.
- This is mostly due to floating point precision error: even though the mathematicla equations of both are same, the operations are not run in the same order, which can lead to small differences.
- The loss since the start of an epoch is the mean of all batch losses seen so far. Each batch loss is the sum of weighted instances divided by the batch size.
- The metric since the start of the epoch is the sum of the weighted instance losses seen so far divided by the sum of weights. It is the weighted mean of all the instance losses.
- Loss = metric * mean of sample weights (plus some floating point precision error factor)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                270       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss = create_huber(2.), optimizer = Nadam(), metrics = [create_huber(2.)])

In [11]:
sample_weights = np.random.rand(len(y_train)) # Creating a sample of weights to be used in the model.

In [12]:
history = model.fit(x_train_scaled, y_train, epochs = 5, sample_weight = sample_weights)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected EOF while parsing (<unknown>, line 1)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected EOF while parsing (<unknown>, line 1)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected EOF while parsing (<unknown>, line 1)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the b

In [18]:
history.history['loss'][0], history.history['huber_loss'][0] * sample_weights.mean() # Loss = metric * mean of sample weights

(0.44554394483566284, 0.44320641348494266)

## Streaming metrics

In [20]:
precision = keras.metrics.Precision() # Creating a precision metric
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1]) # Giving it the labels and predicted values of the first batch.

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [21]:
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0]) # Giving the metric the labels and the predicted values of the second batch.

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [22]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [24]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [25]:
precision.reset_states() # Resetting the metric

In [26]:
class Huber_metric(keras.metrics.Metric):
    def __init__(self, threshold = 1., **kwargs):
        super().__init__(**kwargs) # Handles kwargs of base class
        self.threshold = threshold
        self.total = self.add_weight('total', initializer = 'zeros') # variable for storing the total metric value for all the batches.
        self.count = self.add_weight('count', initializer = 'zeros') # variable for storing the no. of instances
    def huber_fn(self, y_true, y_pred): # Func for cacculating huber loss
        error = y_true - y_pred
        small_loss = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - (tf.square(self.threshold) / 2)
        return tf.where(small_loss, squared_loss, linear_loss)
    def update_state(self, y_true, y_pred, sample_weight = None):
        metric = self.huber_fn(y_true, y_pred) # Huber loss is calculated
        self.total.assign_add(tf.reduce_sum(metric)) # The sum of the metrics of all the instances is added to the variable
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32)) # The total no. of instances is added to the variable
    def result(self):
        return self.total / self.count # The Huber metric over all the instances.
        # When we use the metrics func, first the update_state method is called then the result method and the output is returned.
    def get_config(self): # Method to save the threshold along with the model
        base_config = super().get_config() # Config params from the base class
        return {**base_config, 'threshold' : self.threshold}

In [39]:
m = Huber_metric(2.)

In [40]:
m(tf.constant([[2.]]), tf.constant([[10.]]))
# total = 2 * |10 - 2| - 2^2 / 2 = 14
# count = 1
# metric = total / count = 14

<tf.Tensor: shape=(), dtype=float32, numpy=14.0>

In [41]:
m(tf.constant([[0.], [5.]]), tf.constant([[1.], [9.25]]))
m.result()
# total = total + (|1 - 0|² / 2) + (2 * |9.25 - 5| - 2² / 2) = 14 + 7 = 21
# count = count + 2 = 3
# result = total / count = 21 / 3 = 7

<tf.Tensor: shape=(), dtype=float32, numpy=7.0>

In [42]:
m.variables

[<tf.Variable 'total:0' shape=() dtype=float32, numpy=21.0>,
 <tf.Variable 'count:0' shape=() dtype=float32, numpy=3.0>]

In [43]:
m.reset_states()
m.variables

[<tf.Variable 'total:0' shape=() dtype=float32, numpy=0.0>,
 <tf.Variable 'count:0' shape=() dtype=float32, numpy=0.0>]

In [44]:
pre_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                270       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [45]:
model.compile(loss = create_huber(2.), optimizer = Nadam(), metrics = [Huber_metric(2.)])

In [46]:
history = model.fit(x_train_scaled, y_train, epochs = 5, validation_data = (x_valid_scaled, y_valid))

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected EOF while parsing (<unknown>, line 1)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected EOF while parsing (<unknown>, line 1)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
363/363 [==============================] - 0s 1ms/step - loss: 0.2237 - huber_metric: 0.2237 - val_loss: 0.2958 - val_huber_metric: 0.2958
Epoch 2/5
363/363 [==============================] - 0s 1ms/step - loss: 0.2196 - huber_metric: 0.2196 - val_loss: 0.2754 - val_huber_metric: 0.2754
Epoch 3/5
363/363 [==============================] - 0s 1ms/step - loss: 0.2164 - 

In [47]:
model.save('Custom_huber_metric_model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: Custom_huber_metric_model/assets


In [51]:
model.metrics_names

['loss', 'huber_metric']

In [52]:
model.metrics[-1].threshold

2.0

- We can create a more simple custom metric using the keras.metrics.mean class

In [69]:
class Huber_metric_mean(keras.metrics.Mean):
    def __init__(self, threshold = 1., name = 'Huber_metric_mean', dtype = None):
        self.threshold = threshold
        self.huber_fn = create_huber(threshold) # Creating the huber metric func
        super().__init__(name = name, dtype = dtype)
    def update_state(self, y_true, y_pred, sample_weight = None):
        metric = self.huber_fn(y_true, y_pred) # Calculating huber metric for each instance
        super(Huber_metric_mean, self).update_state(metric, sample_weight) # Recursively updating the metric value and the sample_weights
    def get_config(self):
        base_config = super().get_config() # Getting the config params from the base class
        return {**base_config, 'threshold' : self.threshold} # Saving threshold along with the rest of the params

In [70]:
pre_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                270       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [71]:
model.compile(loss = create_huber(2.), optimizer = Nadam(), metrics = [Huber_metric_mean(2.)])

In [72]:
sample_weight = np.random.rand(len(y_train))
history = model.fit(x_train_scaled, y_train, epochs = 5, validation_data = (x_valid_scaled, y_valid), sample_weight = sample_weight)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected EOF while parsing (<unknown>, line 1)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected EOF while parsing (<unknown>, line 1)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected EOF while parsing (<unknown>, line 1)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the b

In [73]:
history.history['loss'][0], history.history['Huber_metric_mean'][0] * sample_weight.mean()

(0.09483620524406433, 0.09570989377475683)

In [74]:
model.save('Custom_huber_metric_model_v2')

INFO:tensorflow:Assets written to: Custom_huber_metric_model_v2/assets


In [77]:
model.metrics[-1].threshold

2.0